# Hybrid Recommender System

   * **User Based Recommender** 
   
   
   1. Preparation of Data Set
   

   2. Determining the Movies Watched by the User to Make a Suggestion
   

   3. Accessing Data and Ids of Other Users Watching the Same Movies
   

   4. Identifying Users with the Most Similar Behaviors to the User to Suggest
   

   5. Calculating the Weighted Average Recommendation Score
   
   
   * **Item Based Recommender**
   
   
   1. Data Set Preparation
   

   2. Creating the User Movie Df
   

   3. Making Item-Based Movie Recommender
   

## Import Necessary Libraries

In [37]:
import pandas as pd
pd.set_option("display.max_columns",None)
pd.set_option("display.width",500)

## Create the user_movie_df

In [38]:
def create_user_movie_df():
    movie = pd.read_csv("C://Users//Tohid//Downloads//datasets//movie.csv")
    rating = pd.read_csv("C://Users//Tohid//Downloads//datasets//rating.csv")
    df = movie.merge(rating,how="left",on="movieId")
    comment_counts = pd.DataFrame(df["title"].value_counts())
    rare_movies = comment_counts[comment_counts["title"]<=10000].index
    common_movies = df[~df["title"].isin(rare_movies)]
    user_movie_df = common_movies.pivot_table(index=["userId"],columns=["title"],values="rating")
    return user_movie_df

In [39]:
user_movie_df = create_user_movie_df()

## Create User Based Recommender

In [42]:
def user_based_recommender(random_user, user_movie_df, ratio=60, corr_th=0.65, score=3.5):
    random_user_df = user_movie_df[user_movie_df.index==random_user]
    movies_watched = random_user_df.columns[random_user_df.notna().any()].tolist()
    movies_watched_df = user_movie_df[movies_watched]
    user_movie_count = movies_watched_df.T.notnull().sum()
    user_movie_count = user_movie_count.reset_index()
    user_movie_count.columns = ["userId","movie_count"]
    prec = len(movies_watched)*ratio/100
    user_same_movies = user_movie_count[user_movie_count["movie_count"]>prec]["userId"]
    final_df = pd.concat([movies_watched_df[movies_watched_df.index.isin(user_same_movies)],random_user_df[movies_watched]])
    corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()
    corr_df = pd.DataFrame(corr_df,columns=["corr"])
    corr_df.index.names = ["user_id_1","user_id_2"]
    corr_df = corr_df.reset_index()
    top_users = corr_df[(corr_df["user_id_1"]==random_user) & (corr_df["corr"]>=corr_th)][["user_id_2","corr"]].reset_index(drop=True)
    top_users = top_users.sort_values(by="corr",ascending=False)
    top_users.rename(columns={"user_id_2":"userId"},inplace=True)
    rating = pd.read_csv("C://Users//Tohid//Downloads//datasets//rating.csv")
    top_users_ratings = top_users.merge(rating[["userId","movieId","rating"]],how="inner")
    top_users_ratings["weighted_rating"] = top_users_ratings["corr"] * top_users_ratings["rating"]
    recommendation_df = top_users_ratings.groupby(["movieId"]).agg({"weighted_rating":"mean"})
    recommendation_df = recommendation_df.reset_index()
    movies_to_be_rocommend = recommendation_df[recommendation_df["weighted_rating"]>score].sort_values("weighted_rating",ascending=False)
    movie = pd.read_csv("C://Users//Tohid//Downloads//datasets//movie.csv")
    return movies_to_be_rocommend.merge(movie[["movieId","title"]])

In [43]:
random_user = int(pd.Series(user_movie_df.index).sample(1).values)
user_based_recommender(random_user,user_movie_df)

,movieId,weighted_rating,title
0,7398,4.210308,Belles on Their Toes (1952)
1,4836,4.210308,"Competition, The (1980)"
2,114662,3.844475,American Sniper (2014)
3,105844,3.844475,12 Years a Slave (2013)
4,5611,3.844475,"Four Feathers, The (2002)"
5,5498,3.836023,Red Beard (Akahige) (1965)
6,7925,3.836023,"Hidden Fortress, The (Kakushi-toride no san-ak..."
7,6985,3.836023,"Passion of Joan of Arc, The (Passion de Jeanne..."
8,5971,3.836023,My Neighbor Totoro (Tonari no Totoro) (1988)
9,2927,3.749947,Brief Encounter (1946)


## Create Item Based Recommender

In [40]:
def item_based_recommender(movie_name,user_movie_df):
    movie_name = user_movie_df[movie_name]
    return user_movie_df.corrwith(movie_name).sort_values(ascending=False).head(10)

In [41]:
movie_name = pd.Series(user_movie_df.columns).sample(1).values[0]
item_based_recommender(movie_name,user_movie_df)

title
Wedding Singer, The (1998)                      1.000000
Happy Gilmore (1996)                            0.475830
Liar Liar (1997)                                0.373356
American Pie (1999)                             0.354508
Meet the Parents (2000)                         0.347920
Tommy Boy (1995)                                0.344389
Wayne's World (1992)                            0.335576
Father of the Bride Part II (1995)              0.332505
Austin Powers: The Spy Who Shagged Me (1999)    0.329215
10 Things I Hate About You (1999)               0.326827
dtype: float64